In [1]:
import numpy as np
import pandas as pd
from jqdatasdk import *
# import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import sklearn.metrics as metrics
import sklearn.ensemble as se
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
import imblearn.over_sampling as imos
from collections import Counter
from sklearn.ensemble import IsolationForest

from sklearn.ensemble import AdaBoostClassifier 

from pyod.models.iforest import IForest
from pyod.models.copod import COPOD

import math


from imbalanced_ensemble.ensemble import SelfPacedEnsembleClassifier
from imbalanced_ensemble.ensemble import EasyEnsembleClassifier
from imbalanced_ensemble.ensemble import BalanceCascadeClassifier
from imbalanced_ensemble.ensemble import RUSBoostClassifier
from imbalanced_ensemble.ensemble import SMOTEBoostClassifier
from imbalanced_ensemble.ensemble import UnderBaggingClassifier
from imbalanced_ensemble.ensemble import OverBaggingClassifier

from imblearn.combine import SMOTEENN
import smote_variants as sv
from imblearn.datasets import fetch_datasets

提示：当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口 panel 参数将固定为 False
注意：0.25 以上版本 pandas 不支持 panel，如使用该数据结构和相关函数请注意修改


In [2]:
protein_homo = fetch_datasets()["thyroid_sick"]
X, y = protein_homo.data, protein_homo.target
X = pd.DataFrame(X)
y  = pd.DataFrame(y)
y.columns = ["Class"]
data = pd.concat([X,y],axis=1)


# for i in data.columns.values.tolist():
#     mean = data[i].mean()
#     data[i][data[i].isnull()] = mean

negative_data = data.where(data['Class'] == 1).dropna()
positive_data = data.where(data['Class'] == -1).dropna()

data_labels = data[['Class']]
data_features = data.drop(columns=['Class'])


tmp = data_labels.values.tolist()
tmp = np.ravel(tmp)
print('Original dataset shape %s' % Counter(tmp))

train_features, test_features, train_labels, test_labels = train_test_split(
    data_features, data_labels, test_size=0.2)


train_features_res, train_labels_res = SMOTE().fit_resample(train_features, train_labels)


Original dataset shape Counter({-1: 3541, 1: 231})


In [5]:

n_estimators = 50
models = []
weights = []
for i in range (0,n_estimators+1):
    clf_name = 'iforest'

    c = 0.05 + 0.35*(i/n_estimators)
    dis = []
    for j in range(0,5):
        dis.append(1/abs(j/5-1/20-(1-c)))

    clf = IForest(contamination = c) # 初始化检测器clf
    clf.fit(train_features) # 使用X_train训练检测器clf 
    y_train_pred = clf.labels_  # 返回训练数据上的分类标签 (0: 正常值, 1: 异常值)
    y_train_scores = clf.decision_scores_  # 返回训练数据上的异常值 (分值越大越异常)


    num_out = 0
    num_pos = 0
    num_neg = 0
    tmp_train_labels = train_labels.values.tolist()
    for j in range(0,len(y_train_pred)):
        if tmp_train_labels[j][0] - y_train_pred[j] == 1:
            num_out += 1
        if tmp_train_labels[j][0] == -1 and y_train_pred[j] == 1:
            num_neg += 1
        if tmp_train_labels[j][0] == 1 and y_train_pred[j] == 1:
            num_pos += 1
        
    if num_neg == 0:
        num_neg += 1
    if num_pos == 0:
        num_pos += 1
    #print(num_out)
    snr = math.log(negative_data.shape[0]/num_out+1)
    #print(snr)
    
    entropy = -1 * (num_pos/(num_pos+num_neg)*math.log(num_pos/(num_pos+num_neg)) + num_neg/(num_pos+num_neg)*math.log(num_neg/(num_pos+num_neg)))
    #print(entropy)
    #print(entropy*snr)
    weights.append(entropy*snr)



    minmax = MinMaxScaler(feature_range=(0, 1))  # 自动将dtype转换成float64
    y_train_scores = minmax.fit_transform(y_train_scores.reshape(-1,1))

    tmp_train_features = train_features
    
    
    tmp = pd.DataFrame(y_train_scores)
    tmp.columns = ['scores']
    tmp = tmp.assign(bin_labels=pd.cut(tmp['scores'], [-1,0.2,0.4,0.6,0.8, float('inf')], labels=['bin0','bin1','bin2','bin3','bin4']))
    train_data = tmp_train_features.reset_index()
    train_data = pd.concat([train_data,train_labels],axis=1)
    train_data = pd.concat([train_data,tmp],axis=1)
    
    counts = train_data['bin_labels'].value_counts()

    bin0 = train_data.where(train_data['bin_labels']=='bin0').dropna()
    bin1 = train_data.where(train_data['bin_labels']=='bin1').dropna()
    bin2 = train_data.where(train_data['bin_labels']=='bin2').dropna()
    bin3 = train_data.where(train_data['bin_labels']=='bin3').dropna()
    bin4 = train_data.where(train_data['bin_labels']=='bin4').dropna()
    

    resample_data = bin4
    resample_data = resample_data.drop(index=resample_data.index)

    t_sum = 0
    dis = []
    for j in range(0,5):
        t_sum += math.log(1/counts[j])
    
    for j in range(0,5):
        # 超参：20
        if int(negative_data.shape[0]*10*    (math.log(1/counts[j]) /t_sum)) > len(eval('bin' + str(j))):
            resample_data.append(eval('bin' + str(j)))
        else:
            resample_data = resample_data.append(eval('bin' + str(j)).sample(int(negative_data.shape[0]*20*    (math.log(1/counts[j]) /t_sum)),replace=True))
    

    _train_labels = resample_data[['Class']]
    _train_labels = _train_labels.append(negative_data[['Class']])
    _train_features = resample_data.drop(columns=['Class','bin_labels','scores','index'])
    _train_features = _train_features.append(negative_data.drop(columns=['Class']))

    model = DecisionTreeClassifier(max_depth=10).fit(_train_features, _train_labels)
    models.append(model)

In [12]:
pro = [0 for index in range(0,len(test_labels))]

num_model = 0
total_weight = 0
for model in models:
    _predict_proba = model.predict_proba(test_features)
    #print(_predict_proba)
    for i in range(0,len(test_labels)):
        pro[i] += _predict_proba[i][1]*weights[num_model]
    
    total_weight += weights[num_model]
    num_model += 1

    
for i in range(0,len(test_labels)):
    if pro[i]/total_weight>=0.5:
        pro[i] = 1
    else:
        pro[i] = -1

results = []
_predict_labels = pro

accuracy = metrics.accuracy_score(test_labels, _predict_labels)
precision = metrics.precision_score(test_labels,_predict_labels)
recall = metrics.recall_score(test_labels,_predict_labels)
f1_score = metrics.f1_score(test_labels,_predict_labels)
roc_auc = metrics.roc_auc_score(test_labels, _predict_labels)

print("my-Accuracy:" + str(accuracy))
print("my-Precision:" + str(precision))
print("my-Recall:" + str(recall))
print("my-AUC:" + str(roc_auc))
print("my-f1_score:" + str(f1_score))
results.append(["my,",accuracy,precision,recall,roc_auc,f1_score])
    
    

my-Accuracy:0.9920529801324504
my-Precision:0.8604651162790697
my-Recall:1.0
my-AUC:0.9958217270194987
my-f1_score:0.9249999999999999


In [84]:
def randomforest(_train_features, _train_labels, _test_features, _test_labels):
    model = se.RandomForestClassifier(max_depth=10).fit(_train_features, _train_labels)
    _predict_labels = model.predict(_test_features)
    accuracy = metrics.accuracy_score(_test_labels, _predict_labels)
    precision = metrics.precision_score(_test_labels,_predict_labels)
    recall = metrics.recall_score(_test_labels,_predict_labels)
    f1_score = metrics.f1_score(_test_labels,_predict_labels)
    roc_auc = metrics.roc_auc_score(_test_labels, _predict_labels)
    print("RF-Accuracy:" + str(accuracy))
    print("RF-Precision:" + str(precision))
    print("RF-Recall:" + str(recall))
    print("RF-AUC:" + str(roc_auc))
    print("RF-f1_score:" + str(f1_score))
    ans = ['randomforest',accuracy,precision,recall,roc_auc,f1_score]
    return ans

def decisiontree(_train_features, _train_labels, _test_features, _test_labels):
    model = DecisionTreeClassifier(max_depth=10).fit(_train_features, _train_labels)
    _predict_labels = model.predict(_test_features)
    accuracy = metrics.accuracy_score(_test_labels, _predict_labels)
    precision = metrics.precision_score(_test_labels,_predict_labels)
    recall = metrics.recall_score(_test_labels,_predict_labels)
    f1_score = metrics.f1_score(_test_labels,_predict_labels)
    roc_auc = metrics.roc_auc_score(_test_labels, _predict_labels)

    print("DT-Accuracy:" + str(accuracy))
    print("DT-Precision:" + str(precision))
    print("DT-Recall:" + str(recall))
    print("DT-AUC:" + str(roc_auc))
    print("DT-f1_score:" + str(f1_score))
    ans = ['decisiontree',accuracy,precision,recall,roc_auc,f1_score]
    return ans


def gbdt(_train_features, _train_labels, _test_features, _test_labels):
    model = GradientBoostingClassifier(max_depth=10).fit(_train_features, _train_labels)
    _predict_labels = model.predict(_test_features)
    accuracy = metrics.accuracy_score(_test_labels, _predict_labels)
    precision = metrics.precision_score(_test_labels, _predict_labels)
    recall = metrics.recall_score(_test_labels, _predict_labels)
    f1_score = metrics.f1_score(_test_labels, _predict_labels)
    roc_auc = metrics.roc_auc_score(_test_labels, _predict_labels)
    report = metrics.classification_report(_test_labels, _predict_labels)
    print("GBDT-Accuracy:" + str(accuracy))
    print("GBDT-Precision:" + str(precision))
    print("GBDT-Recall:" + str(recall))
    print("GBDT-AUC:" + str(roc_auc))
    print("GBDT-f1_score:" + str(f1_score))
    ans = ['GBDT',accuracy,precision,recall,roc_auc,f1_score]
    return ans

def adaboost(_train_features, _train_labels, _test_features, _test_labels):
    model = AdaBoostClassifier().fit(_train_features, _train_labels)
    _predict_labels = model.predict(_test_features)
    accuracy = metrics.accuracy_score(_test_labels, _predict_labels)
    precision = metrics.precision_score(_test_labels, _predict_labels)
    recall = metrics.recall_score(_test_labels, _predict_labels)
    f1_score = metrics.f1_score(_test_labels, _predict_labels)
    roc_auc = metrics.roc_auc_score(_test_labels, _predict_labels)
    report = metrics.classification_report(_test_labels, _predict_labels)
    print("Adaboost-Accuracy:" + str(accuracy))
    print("Adaboost-Precision:" + str(precision))
    print("GBAdaboostDT-Recall:" + str(recall))
    print("Adaboost-AUC:" + str(roc_auc))
    print("Adaboost-f1_score:" + str(f1_score))
    ans = ['adaboost',accuracy,precision,recall,roc_auc,f1_score]
    return ans

def SPE(_train_features, _train_labels, _test_features, _test_labels):
    model = SelfPacedEnsembleClassifier(base_estimator=DecisionTreeClassifier(max_depth=10)).fit(_train_features, _train_labels)
    _predict_labels = model.predict(_test_features)
    accuracy = metrics.accuracy_score(_test_labels, _predict_labels)
    precision = metrics.precision_score(_test_labels,_predict_labels)
    recall = metrics.recall_score(_test_labels,_predict_labels)
    f1_score = metrics.f1_score(_test_labels,_predict_labels)
    roc_auc = metrics.roc_auc_score(_test_labels, _predict_labels)

    print("SPE-Accuracy:" + str(accuracy))
    print("SPE-Precision:" + str(precision))
    print("SPE-Recall:" + str(recall))
    print("SPE-AUC:" + str(roc_auc))
    print("SPE-f1_score:" + str(f1_score))
    ans = ['SPE',accuracy,precision,recall,roc_auc,f1_score]
    return ans

def BalanceCascade(_train_features, _train_labels, _test_features, _test_labels):
    model = BalanceCascadeClassifier(base_estimator=DecisionTreeClassifier(max_depth=10)).fit(_train_features, _train_labels)
    _predict_labels = model.predict(_test_features)
    accuracy = metrics.accuracy_score(_test_labels, _predict_labels)
    precision = metrics.precision_score(_test_labels, _predict_labels)
    recall = metrics.recall_score(_test_labels, _predict_labels)
    f1_score = metrics.f1_score(_test_labels, _predict_labels)
    roc_auc = metrics.roc_auc_score(_test_labels, _predict_labels)
    report = metrics.classification_report(_test_labels, _predict_labels)
    print("BalanceCascade-Accuracy:" + str(accuracy))
    print("BalanceCascade-Precision:" + str(precision))
    print("BalanceCascade-Recall:" + str(recall))
    print("BalanceCascade-AUC:" + str(roc_auc))
    print("BalanceCascade-f1_score:" + str(f1_score))
    ans = ['BalanceCascade',accuracy,precision,recall,roc_auc,f1_score]
    return ans

def EasyEnsemble(_train_features, _train_labels, _test_features, _test_labels):
    model = EasyEnsembleClassifier(base_estimator=DecisionTreeClassifier(max_depth=10)).fit(_train_features, _train_labels)
    _predict_labels = model.predict(_test_features)
    accuracy = metrics.accuracy_score(_test_labels, _predict_labels)
    precision = metrics.precision_score(_test_labels, _predict_labels)
    recall = metrics.recall_score(_test_labels, _predict_labels)
    f1_score = metrics.f1_score(_test_labels, _predict_labels)
    roc_auc = metrics.roc_auc_score(_test_labels, _predict_labels)
    report = metrics.classification_report(_test_labels, _predict_labels)
    print("EasyEnsemble-Accuracy:" + str(accuracy))
    print("EasyEnsemble-Precision:" + str(precision))
    print("EasyEnsemble-Recall:" + str(recall))
    print("EasyEnsemble-AUC:" + str(roc_auc))
    print("EasyEnsemble-f1_score:" + str(f1_score))
    ans = ['EasyEnsemble',accuracy,precision,recall,roc_auc,f1_score]
    return ans

def RUSBoost(_train_features, _train_labels, _test_features, _test_labels):
    model = RUSBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=10)).fit(_train_features, _train_labels)
    _predict_labels = model.predict(_test_features)
    accuracy = metrics.accuracy_score(_test_labels, _predict_labels)
    precision = metrics.precision_score(_test_labels, _predict_labels)
    recall = metrics.recall_score(_test_labels, _predict_labels)
    f1_score = metrics.f1_score(_test_labels, _predict_labels)
    roc_auc = metrics.roc_auc_score(_test_labels, _predict_labels)
    report = metrics.classification_report(_test_labels, _predict_labels)
    print("RUSBoost-Accuracy:" + str(accuracy))
    print("RUSBoost-Precision:" + str(precision))
    print("RUSBoost-Recall:" + str(recall))
    print("RUSBoost-AUC:" + str(roc_auc))
    print("RUSBoost-f1_score:" + str(f1_score))
    ans = ['RUSBoost',accuracy,precision,recall,roc_auc,f1_score]
    return ans


def SMOTEBOOST(_train_features, _train_labels, _test_features, _test_labels):
    model = SMOTEBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=10)).fit(_train_features, _train_labels)
    _predict_labels = model.predict(_test_features)
    accuracy = metrics.accuracy_score(_test_labels, _predict_labels)
    precision = metrics.precision_score(_test_labels, _predict_labels)
    recall = metrics.recall_score(_test_labels, _predict_labels)
    f1_score = metrics.f1_score(_test_labels, _predict_labels)
    roc_auc = metrics.roc_auc_score(_test_labels, _predict_labels)
    report = metrics.classification_report(_test_labels, _predict_labels)
    print("SMOTEBOOST-Accuracy:" + str(accuracy))
    print("SMOTEBOOST-Precision:" + str(precision))
    print("SMOTEBOOST-Recall:" + str(recall))
    print("SMOTEBOOST-AUC:" + str(roc_auc))
    print("SMOTEBOOST-f1_score:" + str(f1_score))
    ans = ['SMOTEBOOST',accuracy,precision,recall,roc_auc,f1_score]
    return ans

def UnderBagging(_train_features, _train_labels, _test_features, _test_labels):
    model = UnderBaggingClassifier().fit(_train_features, _train_labels)
    _predict_labels = model.predict(_test_features)
    accuracy = metrics.accuracy_score(_test_labels, _predict_labels)
    precision = metrics.precision_score(_test_labels, _predict_labels)
    recall = metrics.recall_score(_test_labels, _predict_labels)
    f1_score = metrics.f1_score(_test_labels, _predict_labels)
    roc_auc = metrics.roc_auc_score(_test_labels, _predict_labels)
    report = metrics.classification_report(_test_labels, _predict_labels)
    print("UnderBagging-Accuracy:" + str(accuracy))
    print("UnderBagging-Precision:" + str(precision))
    print("UnderBagging-Recall:" + str(recall))
    print("UnderBagging-AUC:" + str(roc_auc))
    print("UnderBagging-f1_score:" + str(f1_score))
    ans = ['UnderBagging',accuracy,precision,recall,roc_auc,f1_score]
    return ans


def OverBagging(_train_features, _train_labels, _test_features, _test_labels):
    model = OverBaggingClassifier().fit(_train_features, _train_labels)
    _predict_labels = model.predict(_test_features)
    accuracy = metrics.accuracy_score(_test_labels, _predict_labels)
    precision = metrics.precision_score(_test_labels, _predict_labels)
    recall = metrics.recall_score(_test_labels, _predict_labels)
    f1_score = metrics.f1_score(_test_labels, _predict_labels)
    roc_auc = metrics.roc_auc_score(_test_labels, _predict_labels)
    report = metrics.classification_report(_test_labels, _predict_labels)
    print("OverBagging-Accuracy:" + str(accuracy))
    print("OverBagging-Precision:" + str(precision))
    print("OverBagging-Recall:" + str(recall))
    print("OverBagging-AUC:" + str(roc_auc))
    print("OverBagging-f1_score:" + str(f1_score))
    ans = ['OverBagging',accuracy,precision,recall,roc_auc,f1_score]
    return ans




results.append(decisiontree(train_features_res, train_labels_res, test_features, test_labels))


results.append(decisiontree(train_features, train_labels, test_features, test_labels))
results.append(randomforest(train_features, train_labels, test_features, test_labels))
results.append(gbdt(train_features, train_labels, test_features, test_labels))
results.append(adaboost(train_features, train_labels, test_features, test_labels))
results.append(SPE(train_features, train_labels, test_features, test_labels))
results.append(BalanceCascade(train_features, train_labels, test_features, test_labels))
results.append(EasyEnsemble(train_features, train_labels, test_features, test_labels))
results.append(RUSBoost(train_features, train_labels, test_features, test_labels))
results.append(SMOTEBOOST(train_features, train_labels, test_features, test_labels))
results.append(UnderBagging(train_features, train_labels, test_features, test_labels))
results.append(OverBagging(train_features, train_labels, test_features, test_labels))


results = pd.DataFrame(results)
results.to_csv("abalone.csv")

DT-Accuracy:0.80622009569378
DT-Precision:0.29207920792079206
DT-Recall:0.7564102564102564
DT-AUC:0.7838779514241255
DT-f1_score:0.42142857142857143
DT-Accuracy:0.8863636363636364
DT-Precision:0.09523809523809523
DT-Recall:0.02564102564102564
DT-AUC:0.5002875312901698
DT-f1_score:0.0404040404040404


<ipython-input-84-d549d4dc7f28>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = se.RandomForestClassifier(max_depth=10).fit(_train_features, _train_labels)


RF-Accuracy:0.9055023923444976
RF-Precision:0.4
RF-Recall:0.02564102564102564
RF-AUC:0.5108416209999322
RF-f1_score:0.048192771084337345


C:\Users\UNVS\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GBDT-Accuracy:0.8863636363636364
GBDT-Precision:0.2702702702702703
GBDT-Recall:0.1282051282051282
GBDT-AUC:0.5462925377173399
GBDT-f1_score:0.17391304347826086
Adaboost-Accuracy:0.9055023923444976
Adaboost-Precision:0.4
GBAdaboostDT-Recall:0.02564102564102564
Adaboost-AUC:0.5108416209999322
Adaboost-f1_score:0.048192771084337345


...

SMOTEBOOST-Accuracy:0.8696172248803827
SMOTEBOOST-Precision:0.34951456310679613
SMOTEBOOST-Recall:0.46153846153846156
SMOTEBOOST-AUC:0.6865739801096002
SMOTEBOOST-f1_score:0.3977900552486188


C:\Users\UNVS\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


UnderBagging-Accuracy:0.8026315789473685
UnderBagging-Precision:0.3013698630136986
UnderBagging-Recall:0.8461538461538461
UnderBagging-AUC:0.822153440227319
UnderBagging-f1_score:0.4444444444444444


C:\Users\UNVS\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


OverBagging-Accuracy:0.8995215311004785
OverBagging-Precision:0.375
OverBagging-Recall:0.11538461538461539
OverBagging-AUC:0.5477978485894054
OverBagging-f1_score:0.1764705882352941
